# 05-3 트리의 앙상블
앙상블 학습이 무엇인지 이해하고 다양한 앙상블 학습 알고리즘을 실습을 통해 배운다.

## 정형 데이터와 비정형 데이터
- 정형 데이터 : 정해진 형식과 구조에 따라 저장된 데이터
- 비정형 데이터 : 텍스트 데이터, 사진, 음악 등


**정형 데이터**에는 머신러닝 알고리즘이 잘 맞음. 그중에 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 것이 앙상블 학습이다.

참고로 **비정형 데이터**에는 신경망 알고리즘이 좋음. 왜냐면 비정형 데이터는 규칙성을 찾기 어려워 전통적인 머신러닝 방법으로는 모델을 만들기 까다롭다.

## 랜덤포레스트
- 부트스트랩 샘플 사용. 앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕분에 널리 사용되고 있음
- 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든다. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만든다.

랜덤포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고, 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.

여기서 '**부트스트랩 방식**'이란 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식을 뜻한다.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
# 기본적으로 100개의 결정 트리 사용
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
# return_train_score 매개변수가 True면, 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환됨

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)

print(rf.feature_importances_)
# 결정 트리에서의 특성 중요도와 차이가 있음
# 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문
# 하나의 특성에 과도하게 집중하지 않고, 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다 → 과대적합 줄이고, 일반화 성능 높이는데 도움

[0.23167441 0.50039841 0.26792718]


In [4]:
# RandomForestClassifier에서는 자체적으로 모델을 평가하는 점수를 얻을 수 있음
# OOB 점수를 사용하면 교차 검증을 대신할 수 있어서 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있음
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)

print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리
엑스트라 트리는 랜덤 포레스트와 비슷하게 결정 트리를 사용하여 앙상블 모델을 만들지만 부트스트랩 샘플을 사용하지 않음. 대신 랜덤하게 노드를 분할해 과대적합을 감소시킨다.

랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점

In [5]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)

print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
- 랜덤 포레스트나 엑스트라 트리와 달리 결정 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방법. 이런 이유로 훈련 속도가 조금 느리지만 더 좋은 성능을 기대할 수 있음

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
import time
start = time.time()
# 학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음
# learning_rate : 학습률, 트리가 앙상블에 기여하는 정도(기본값 0.1)
# n_estimators : 결정 트리 개수(기본값 100)
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print("time :", time.time() - start)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

time : 9.986716747283936
0.9464595437171814 0.8780082549788999


In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 그레이디언트 부스팅
가장 뛰어난 앙상블 학습으로 평가받는다.

히스토그램 기반 그레이디언트 부스팅은 훈련 데이터를 256개의 구간으로 변환하여 사용하기 때문에 노드 분할 속도가 매우 빠름

In [10]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


0.9321723946453317 0.8801241948619236


In [11]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [12]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [13]:
hgb.score(test_input, test_target)

0.8723076923076923

### **XGBoost** 라이브러리에서의 히스토그램 기반 그레이디언트 부스팅 알고리즘 구현

In [14]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


### **LightGBM** 라이브러리에서의 히스토그램 기반 그레이디언트 부스팅 알고리즘 구현

In [15]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
